## TuMag's reduction example

In this file we provide an example of the reduction for an aobserving mode of TuMag starting from a fits file.



2 paths should be identified before starting the demodulation, the local directory where the TuMag's pipeline is located and the fits file to reduce. 

In [7]:
# SUBSTITUTE for local paths
pipeline_path = "/home/pablo/Desktop/TuMag/TuMags_Reduction_Pipeline"
fits_file = "/media/HDD/TuMag_data/obs/01_QSUN_TM_00_Fe2.02_8_10072024T133713_LV_0.6_v1.0.fits"

### Imports

From external librarys:
 - fits from astropy.io

From the directory containing the pipeline, the following routines must be imported: 
 - align_obs_mode from alignment.py
 - demodulate from demodulation.py
 - fit_muller_matrix from xtalk_jaeggli.py

To do this we use the sys library (pyhon built-in)

In [ ]:
# IMPORTS

# External libraries
import sys      
from astropy.io import fits

# TuMag pipeline functions
sys.path.append(pipeline_path) 
from alignment import align_obsmode 
from demodulation import demodulate
from xtalk_jaeggli import fit_mueller_matrix

### Loading the data
We load the fits data file with the _getdata_ routine and the header with the _getheader_ from the module _astropy.io_. 

TuMag's fits files have the following dimensions -> (Ncams, Nlambda, Nmods, Nx, Ny), for data not yet demodulated (reduction levels lower than 1.0).

Some fields of the header are useful for the reduction:
 - Number of wavelengths. 
 - Number of modulations
 - Spectral line. 


In [ ]:
# LOAD the data
obs_data = fits.getdata(fits_file)
obs_info = fits.getheader(fits_file)

nmods = obs_info["NMODS"]
nlambda = obs_info["NLAMBDA"]
line = obs_info["FW2"]
obsmode = obs_info["OBS_MODE"]


print(f"Data shape: {np.shape(obs_data)}")
print(f"Data properties:\nObserving mode: {obsmode} - Spectral line {line}\nNº of Modulations : {nmods}\nNº of Wavelengths {nlambda}")

Data shape: (2, 8, 4, 1644, 1644)
Data properties:
Observing mode: 2.02 - Spectral line 525.02
Nº of Modulations : 4
Nº of Wavelengths 8


### Filter and alignment 

Since the fits file we employed has a reduction level lower than 0.8 we need to align and filter the data before demodulating. 

We are going to use the  __align_obsmode__ routine from the module _alignment.py_. 

In [10]:
# ALIGN the data. 
aligned = align_obsmode(obs_data)

Performing noise filtration ...
Procesing wavelength: 1 / 8
Time elapsed: 1.941 s.
Procesing wavelength: 2 / 8
Time elapsed: 1.906 s.
Procesing wavelength: 3 / 8
Time elapsed: 1.899 s.
Procesing wavelength: 4 / 8
Time elapsed: 1.886 s.
Procesing wavelength: 5 / 8
Time elapsed: 1.896 s.
Procesing wavelength: 6 / 8
Time elapsed: 1.9 s.
Procesing wavelength: 7 / 8
Time elapsed: 1.898 s.
Procesing wavelength: 8 / 8
Time elapsed: 1.889 s.
Filtering process completed.

Computing camera 2 rotation...
Rotation finished.

Aligning wavelengh: 1/8
Modulations of cam 1 alignment...


/home/pablo/Desktop/TuMag/TuMags_Reduction_Pipeline/alignment.py:63: RuntimeWarning: invalid value encountered in sqrt
  error=np.sqrt(1-np.abs(CCmax)**2/(rg00*rf00))


Aligning cam2...
mod -> 0...
mod -> 1...
mod -> 2...
mod -> 3...
Aligning wavelengh: 2/8
Modulations of cam 1 alignment...
Aligning cam2...
mod -> 0...
mod -> 1...
mod -> 2...
mod -> 3...
Aligning wavelengh: 3/8
Modulations of cam 1 alignment...
Aligning cam2...
mod -> 0...
mod -> 1...
mod -> 2...
mod -> 3...
Aligning wavelengh: 4/8
Modulations of cam 1 alignment...
Aligning cam2...
mod -> 0...
mod -> 1...
mod -> 2...
mod -> 3...
Aligning wavelengh: 5/8
Modulations of cam 1 alignment...
Aligning cam2...
mod -> 0...
mod -> 1...
mod -> 2...
mod -> 3...
Aligning wavelengh: 6/8
Modulations of cam 1 alignment...
Aligning cam2...
mod -> 0...
mod -> 1...
mod -> 2...
mod -> 3...
Aligning wavelengh: 7/8
Modulations of cam 1 alignment...
Aligning cam2...
mod -> 0...
mod -> 1...
mod -> 2...
mod -> 3...
Aligning wavelengh: 8/8
Modulations of cam 1 alignment...
Aligning cam2...
mod -> 0...
mod -> 1...
mod -> 2...
mod -> 3...


### Demodulating
After alignment, we can demodulate the data to compute the Stokes components with the demodulation routine from the module _demodulation.py_.

In addition to the aligned data, this function needs the spectral line to be able to select the demodulation matrix.

In [11]:
stokes = demodulate(aligned, filt = line)

## Correcting the xtalk. 

After demodulation, a first cross-talk correction is required. This correction can be performed with the __fit_mueller_matrix__ routine from the __xtalk_jaeggli.py__ module

In [12]:
xtalk_corr = fit_mueller_matrix(stokes)